# PART I:

(10 points) Do exercise 3.4 from Chapter 3 in the textbook: [https://web.stanford.edu/~jurafsky/slp3/3.pdf](https://web.stanford.edu/~jurafsky/slp3/3.pdf)

We are given the following corpus, modified from the one in the chapter:

    <s> I am Sam </s>
    <s> Sam I am </s>
    <s> I am Sam </s>
    <s> I do not like green eggs and Sam </s>

Using a bigram language model with add-one smoothing, what is P(Sam | am)? Include \<s> and \</s> in your counts just like any other token.

In [741]:
p1_corpora = filenameToCorpora('p1.txt', startToken='<s>', stopToken='</s>')
p1_unigrams, p1_bigrams = processGrams(p1_corpora)
p1Model = SmoothBigramModel(p1_unigrams, p1_bigrams)
output = p1Model.bigramMLE('sam', 'am', log=False, verbose=True)


P(\texttt{am} \mid \texttt{sam}) &=  \frac{count^{*}(\texttt{am} , \texttt{sam}) + 1}{count(am) + |V|} = \frac{2 + 1}{3 + 10} = 0.23076923076923078  \\


# PART II:

In this assignment, you will train several language models and will evaluate them on a test corpus. You can discuss in groups, but the homework is to be completed and submitted individually. Two files are provided with this assignment:

1. train.txt
2. test.txt

Each file is a collection of texts, one sentence per line. train.txt contains 10,000 sentences from the NewsCrawl corpus. You will use this corpus to train the language models. The test corpus test.txt is from the same domain and will be used to evaluate the language models that you trained.

## 1.1 PRE-PROCESSING

Prior to training, please complete the following pre-processing steps:


### 1. Pad each sentence in the training and test corpora with start and end symbols (you can use \<s> and \</s>, respectively).

In [9]:
def loadSentences(filename):
    with open(filename, 'r') as f:
        corpora = f.read().strip()
        return [ sentence.strip().split(' ') for sentence in corpora.split('\n') ]

In [11]:
train_corpora = loadSentences('data/train.txt')
print('TRAIN', train_corpora[1])
assert len(train_corpora) == 100000, f'train must have 10000 sentences, got {len(train_corpora)} instead'

test_corpora = loadSentences('data/test.txt')
print('TEST', test_corpora[2])
assert len(test_corpora) == 100, f'test must have 100 sentences, got {len(test_corpora)} instead'

TRAIN ['Man', 'charged', 'over', 'drugs', 'seizure']
TEST ['The', 'road', 'was', 'pitted', 'with', 'tank', 'treads', '.']


In [12]:
def padSentence(sentence, startToken='<s>', stopToken='</s>'):
    return [startToken] + sentence + [stopToken]

In [13]:
train_padded_corpora = [ padSentence(sentence) for sentence in train_corpora ]
print('TRAIN', train_corpora[1], '->', train_padded_corpora[1])

test_padded_corpora = [padSentence(sentence) for sentence in test_corpora]
print('TEST', test_corpora[1], '->', test_padded_corpora[2])

TRAIN ['Man', 'charged', 'over', 'drugs', 'seizure'] -> ['<s>', 'Man', 'charged', 'over', 'drugs', 'seizure', '</s>']
TEST ['If', 'you', 'have', 'owned', 'the', 'property', 'for', 'more', 'than', 'three', 'years', ',', 'you', 'can', 'apply', 'for', '"', 'taper', 'relief', ',', '"', 'by', 'which', 'you', 'can', 'reduce', 'any', 'taxable', 'gain', 'by', '5%', 'for', 'each', 'year', 'of', 'ownership', ',', 'up', 'to', 'a', 'maximum', '40%', '.'] -> ['<s>', 'The', 'road', 'was', 'pitted', 'with', 'tank', 'treads', '.', '</s>']


### 2. Lowercase all words in the training and test corpora. Note that the data already has been tokenized (i.e. the punctuation has been split off words).

In [14]:
def lowerSentence(sentence):
    return [ word.lower() for word in sentence ]


In [15]:
train_lowercased_padded_corpora = [ lowerSentence(sentence) for sentence in train_padded_corpora ]
print('TRAIN', train_padded_corpora[1], '->', train_lowercased_padded_corpora[1])
assert ' '.join(train_lowercased_padded_corpora[1]) == '<s> man charged over drugs seizure </s>', f'train_lowercased_padded_corpora[1] must be "<s> man charged over drugs seizure </s>", got {train_lowercased_padded_corpora[1]} instead'
train_lowercased_padded_vocabulary = set(word for sentence in train_lowercased_padded_corpora for word in sentence)

test_lowercased_padded_corpora = [ lowerSentence(sentence) for sentence in test_padded_corpora ]
print('TEST', test_padded_corpora[2], '->', test_lowercased_padded_corpora[2])
assert ' '.join(test_lowercased_padded_corpora[2]) == '<s> the road was pitted with tank treads . </s>', f'test_lowercased_padded_corpora[2] must be "<s> the road was pitted with tank treads . </s>", got {lowercased_padded_test[1]} instead'
lowercased_padded_test_vocabulary = set(word for sentence in test_lowercased_padded_corpora for word in sentence)

TRAIN ['<s>', 'Man', 'charged', 'over', 'drugs', 'seizure', '</s>'] -> ['<s>', 'man', 'charged', 'over', 'drugs', 'seizure', '</s>']
TEST ['<s>', 'The', 'road', 'was', 'pitted', 'with', 'tank', 'treads', '.', '</s>'] -> ['<s>', 'the', 'road', 'was', 'pitted', 'with', 'tank', 'treads', '.', '</s>']


### 3. Replace all words occurring in the training data once with the token \<unk>. Everyword in the test data not seen in training should be treated as \<unk>.

In [16]:
from itertools import tee

def loadSentences(filename):
    with open(filename, 'r') as f:
        corpora = f.read().strip()
        return [ sentence.strip().split(' ') for sentence in corpora.split('\n') ]

def padSentence(sentence, startToken='<s>', stopToken='</s>'):
    return [startToken] + sentence + [stopToken]

def lowerSentence(sentence):
    return [ word.lower() for word in sentence ]

def pairwise(iterable):
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def sentenceToWords(sentence, startToken='<s>', stopToken='</s>', unknownToken='<unk>', unknownWords=set(), knownWords=set(), ignoreWords=set()):
    words = sentence.strip().lower().split(' ')
    isUnknown = lambda word: word in unknownWords or (len(knownWords) > 0 and word not in knownWords)
    words = [ unknownToken if isUnknown(word) else word for word in words ]
    words =  [startToken, *words, stopToken]
    words = [ word for word in words if word not in ignoreWords ]
    return words

def replaceWordsInSentence(sentence, wordsForReplacing=[], replacingToken='<unk>'):
    return [
        word if word not in wordsForReplacing else replacingToken
        for word in sentence
    ]

def filenameToCorpora(filename, startToken='<s>', stopToken='</s>', unknownToken='<unk>', unknownWords=[]):
    return [
        replaceWordsInSentence(
            lowerSentence(
                padSentence(sentence, startToken, stopToken)),
            wordsForReplacing=unknownWords, replacingToken=unknownToken)
        for sentence in loadSentences(filename)
    ]

def processGrams(corpora):
    unigrams = {}
    bigrams = {}
    for sentence in corpora:
        for (condition, word) in pairwise(sentence):
            if condition not in bigrams:
                bigrams[condition] = {}
            if word not in bigrams[condition]:
                bigrams[condition][word] = 0
            bigrams[condition][word] += 1
        for token_type in sentence:
            if token_type not in unigrams:
                unigrams[token_type] = 0
            unigrams[token_type] += 1
    return unigrams, bigrams


In [17]:
train_corpora = filenameToCorpora('data/train.txt', startToken='<s>', stopToken='</s>')
train_unigrams, train_bigrams = processGrams(train_corpora)

In [18]:
print('total sentences', len(train_corpora))

print('total words', sum(len([word for word in sentence if word not in {'<s>', '</s>'}]) for sentence in train_corpora))
print('total words with start/stop', sum(count for count  in train_unigrams.values()))

print('total unigrams types', len([ count for unigram, count in train_unigrams.items() if unigram not in {'<s>', '</s>'} ]))
print('total unigrams types with start/stop', len(train_unigrams))

print('total unigrams words', sum(count for unigram, count in train_unigrams.items() if unigram not in {'<s>', '</s>'}))
print('total unigrams words with start/stop', sum(train_unigrams.values()))

print('total bigrams types with start/stop', len(train_bigrams))
print('total bigrams words with start/stop', sum(len(condition_word.values()) for condition_word in train_bigrams.values()))

total sentences 100000
total words 2368210
total words with start/stop 2568210
total unigrams types 83043
total unigrams types with start/stop 83045
total unigrams words 2368210
total unigrams words with start/stop 2568210
total bigrams types with start/stop 83044
total bigrams words with start/stop 809973


In [21]:
test_corpora = filenameToCorpora('data/test.txt', startToken='<s>', stopToken='</s>')
test_unigrams, test_bigrams = processGrams(test_corpora)

In [22]:
print('total sentences', len(test_corpora))

print('total words', sum(len([word for word in sentence if word not in {'<s>', '</s>'}]) for sentence in test_corpora))
print('total words with start/stop', sum(count for count  in test_unigrams.values()))

print('total unigrams types', len([ count for unigram, count in test_unigrams.items() if unigram not in {'<s>', '</s>'} ]))
print('total unigrams types with start/stop', len(test_unigrams))

print('total unigrams words', sum(count for unigram, count in test_unigrams.items() if unigram not in {'<s>', '</s>'}))
print('total unigrams words with start/stop', sum(test_unigrams.values()))

print('total bigrams types with start/stop', len(test_bigrams))
print('total bigrams words with start/stop', sum(len(condition_word.values()) for condition_word in test_bigrams.values()))

total sentences 100
total words 2669
total words with start/stop 2869
total unigrams types 1247
total unigrams types with start/stop 1249
total unigrams words 2669
total unigrams words with start/stop 2869
total bigrams types with start/stop 1248
total bigrams words with start/stop 2421


In [23]:
test_only_unigrams = { unigram: count for unigram, count in test_unigrams.items() if unigram not in train_unigrams }

test_only_bigrams = {}
for condition, word_count in test_bigrams.items():
    if condition not in train_bigrams:
        test_only_bigrams[condition] = word_count
    else:
        for word, count in word_count.items():
            if word not in train_bigrams[condition]:
                if condition not in test_only_bigrams:
                    test_only_bigrams[condition] = {}
                test_only_bigrams[condition][word] = count

In [25]:
print('only test unigrams types', len(test_only_unigrams.keys()))
print('only test unigrams words', sum(test_only_unigrams.values()))

print('only test bigram types', sum(len(token_count.values()) for token_count in test_only_bigrams.values()))
print('only test bigram words', sum(sum(token_count.values()) for token_count in test_only_bigrams.values()))

only test unigrams types 45
only test unigrams words 46
only test bigram types 722
only test bigram words 725


In [29]:
train_once_unigrams = { word: count for word, count in train_unigrams.items() if count == 1 }

train_corpora_with_replacing = filenameToCorpora('data/train.txt', startToken='<s>', stopToken='</s>', unknownToken='<unk>', unknownWords=set(train_once_unigrams.keys()))
train_unigrams_with_replacing, train_bigrams_with_replacing = processGrams(train_corpora_with_replacing)

In [30]:
print('total sentences', len(train_corpora_with_replacing))

print('total words', sum(len([word for word in sentence if word not in {'<s>', '</s>'}]) for sentence in train_corpora_with_replacing))
print('total words with start/stop', sum(count for count  in train_unigrams_with_replacing.values()))

print('total unigrams types', len([ count for unigram, count in train_unigrams_with_replacing.items() if unigram not in {'<s>', '</s>'} ]))
print('total unigrams types with start/stop', len(train_unigrams_with_replacing))

print('total unigrams words', sum(count for unigram, count in train_unigrams_with_replacing.items() if unigram not in {'<s>', '</s>'}))
print('total unigrams words with start/stop', sum(train_unigrams_with_replacing.values()))

print('total bigrams types with start/stop', len(train_bigrams_with_replacing))
print('total bigrams words with start/stop', sum(len(condition_word.values()) for condition_word in train_bigrams_with_replacing.values()))


total sentences 100000
total words 2368210
total words with start/stop 2568210
total unigrams types 41737
total unigrams types with start/stop 41739
total unigrams words 2368210
total unigrams words with start/stop 2568210
total bigrams types with start/stop 41738
total bigrams words with start/stop 742294


In [32]:
train_once_test_only_unigrams = set(train_once_unigrams.keys()).union(set(test_only_unigrams.keys()))

test_corpora_with_replacing = filenameToCorpora('data/test.txt', startToken='<s>', stopToken='</s>', unknownToken='<unk>', unknownWords=train_once_test_only_unigrams)
test_unigrams_with_replacing, test_bigrams_with_replacing = processGrams(test_corpora_with_replacing)
print(len(test_corpora_with_replacing))
print(len(test_unigrams_with_replacing))
print(sum(test_unigrams_with_replacing.values()))
print(sum(len(condition_word.values()) for condition_word in test_bigrams_with_replacing.values()))
print(sum(sum(condition_word.values()) for condition_word in test_bigrams_with_replacing.values()))

100
1175
2869
2365
2769


## 1.2 TRAINING THE MODELS

Please use train.txt to train the following language models:

### 1. A unigram maximum likelihood model.

In [34]:
import math

class UnigramModel:
    def __init__(self, unigrams, ignoredWords={'<s>'}):
        self.ignoredWords = ignoredWords
        self.unigrams = { unigram: count for unigram, count in unigrams.items() if unigram not in self.ignoredWords }
        self.unigrams_count = sum(self.unigrams.values())
    
    def operatorSumOrProd(self, values, log=False):
        return sum(values) if log else math.prod(values)
    
    def operatorPlusOrTimes(self, log=False):
        return ' + ' if log else ' \\times '
    
    def unigramMLE(self, word, log=False, verbose=False):
        denominator = self.unigrams_count
        if word in self.unigrams:
            numerator = self.unigrams[word]
            wordProbability = numerator / denominator
        else:
            numerator = 0
            wordProbability = 0
        if log:
            wordProbability = math.log(wordProbability, 2) if wordProbability > 0 else -math.inf
        
        steps = [
            'P(\\texttt{'+ word + '})',
            '\\frac{count(\\texttt{'+ word + '})}{count()}',
            '\\frac{' + str(numerator) +  '}{' + str(denominator) + '}'
        ]
        if log:
            steps = [ '\log_{2} (' + step + ')' for step in steps ]
        steps.append(wordProbability)

        if verbose:
            print(steps[0] + ' =&\\ ', ' = '.join([ str(step) for step in steps[1:] ]), ' \\\\')
        return steps

    def sentenceMLE(self, sentence, log=False, verbose=False):
        if verbose:
            print("\\begin{equation}\\begin{split}")
            print('S =&\\ \\texttt{' + sentence + '} \\\\')

        words = sentenceToWords(sentence, knownWords=set(self.unigrams.keys()), ignoreWords=self.ignoredWords)
        wordProbabilities = [ self.unigramMLE(word, log=log, verbose=verbose) for word in words if word not in self.ignoredWords ]
        sentenceProbability = self.operatorSumOrProd([ wordProbability for *_, wordProbability in wordProbabilities ], log=log)
        
        steps = [
            'P(S)',
            'P(\\texttt{'+ sentence + '})',
            'P(' + ', '.join([ '\\texttt{' + word + '}' for word in words]) +')'
        ]
        if log:
            steps = [ '\log_{2} (' + step + ')' for step in steps ]
        steps.append(self.operatorPlusOrTimes(log).join([ wordProbabilityKey for wordProbabilityKey, *_ in wordProbabilities ]))
        steps.append(self.operatorPlusOrTimes(log).join([ str(wordProbability) for *_, wordProbability in wordProbabilities ]))
        steps.append(len(wordProbabilities))
        steps.append(sentenceProbability)

        if verbose:
            print(steps[0], '=&\\', ' =&\\ '.join([f'{step} \\\\' for step in steps[1:]]))
            print("\\end{split}\\end{equation}")
        return steps

### 2. A bigram maximum likelihood model.

In [35]:
import math

class BigramModel(UnigramModel):
    def __init__(self, unigrams, bigrams, ignoreWords={}):
        super().__init__(unigrams, ignoredWords=ignoreWords)
        self.bigrams = bigrams
    
    def bigramMLE(self, word, condition, log=False, verbose=False):
        if condition in self.bigrams and word in self.bigrams[condition]:
            bigramCount = self.bigrams[condition][word]
            unigramCount = self.unigrams[condition]
            conditionalProbability = bigramCount / unigramCount
        else:
            bigramCount = 0
            unigramCount = 0
            conditionalProbability = 0
        if log:
            conditionalProbability = math.log(conditionalProbability, 2) if conditionalProbability > 0 else -math.inf
        steps = [
            'P(\\texttt{' + condition + '} \mid \\texttt{' + word + '})',
            '\\frac{count(\\texttt{' + condition + '} , \\texttt{' + word + '})}{count(' + condition + ')}',
            '\\frac{' + str(bigramCount) +  '}{' + str(unigramCount) + '}'
        ]
        if log:
            steps = [ '\log_{2} (' + step + ')' for step in steps ]
        steps.append(conditionalProbability)
        if verbose:
            print(steps[0] + ' =&\\ ', ' = '.join([ str(step) for step in steps[1:] ]), ' \\\\')
        return steps

    def sentenceMLE(self, sentence, log=False, verbose=False):
        if verbose:
            print("\\begin{equation}\\begin{split}")
            print('S =&\\ \\texttt{' + sentence + '} \\\\')
        
        words = sentenceToWords(sentence, knownWords=set(self.unigrams.keys()), ignoreWords=self.ignoredWords)

        wordProbabilities = [ self.bigramMLE(word, condition, log=log, verbose=verbose) for condition, word in pairwise(words) ]
        sentenceProbability = self.operatorSumOrProd([ wordProbability for *_, wordProbability in wordProbabilities ], log=log)

        steps = [
            'P(S)',
            'P(\\texttt{'+ sentence + '})',
            'P(' + ', '.join([ '\\texttt{' + word + '}' for word in words]) +')'
        ]
        if log:
            steps = [ '\log_{2} (' + step + ')' for step in steps ]
        steps.append(self.operatorPlusOrTimes(log).join([ wordProbabilityKey for wordProbabilityKey, *_ in wordProbabilities ]))
        steps.append(self.operatorPlusOrTimes(log).join([ str(wordProbability) for *_, wordProbability in wordProbabilities ]))
        steps.append(len(wordProbabilities))
        steps.append(sentenceProbability)

        if verbose:
            print(steps[0], '&=', ' =&\\ '.join([f'{step} \\\\' for step in steps[1:]]))
            print("\\end{split}\\end{equation}")
        return steps


### 3. A bigram model with Add-One smoothing.

In [36]:
import math

class SmoothBigramModel(BigramModel):
    def bigramMLE(self, word, condition, log=False, verbose=False):
        if condition in self.bigrams and word in self.bigrams[condition]:
            bigramCount = self.bigrams[condition][word]
        else:
            bigramCount = 0
        if condition in self.unigrams:
            unigramCount = self.unigrams[condition]
        else:
            unigramCount = 0
        vocab_size = len(self.unigrams.keys())
        conditionalProbability = (bigramCount + 1) / (unigramCount + vocab_size)
        if log:
            conditionalProbability = math.log(conditionalProbability, 2) if conditionalProbability > 0 else -math.inf
        steps = [
            'P(\\texttt{' + condition + '} \mid \\texttt{' + word + '})',
            '\\frac{count^{*}(\\texttt{' + condition + '} , \\texttt{' + word + '}) + 1}{count(' + condition + ') + |V|}',
            '\\frac{' + str(bigramCount) +  ' + 1}{' + str(unigramCount) + ' + ' + str(vocab_size) + '}'
        ]
        if log:
            steps = [ '\log_{2} (' + step + ')' for step in steps ]
        steps.append(conditionalProbability)
        if verbose:
            print(steps[0] + ' =&\\ ', ' = '.join([ str(step) for step in steps[1:] ]), ' \\\\')
        return steps



### 4. A bigram model with discounting and Katz backoff. Please use a discount constant of 0.5 (see lecture on smoothing).

In [37]:
class KatzBigramModel(BigramModel):
    def __init__(self, unigrams, bigrams, ignoreWords={}):
        BigramModel.__init__(self, unigrams, bigrams, ignoreWords=ignoreWords)

        self.bigrams_star = {}
        for condition, word_count in self.bigrams.items():
            if condition not in self.bigrams_star:
                self.bigrams_star[condition] = {}
            for word, count in word_count.items():
                if word not in self.bigrams_star[condition]:
                    self.bigrams_star[condition][word] = 0
                self.bigrams_star[condition][word] = count - 0.5
        self.A = {}
        self.B = {}
        self.P_B = {}
        self.P_ML = { word: self.unigramMLE(word)[-1] for word in self.unigrams }
    
    def trainB(self, condition):
        self.A[condition] = {}
        self.B[condition] = {}
        for unigram in self.unigrams:
            if condition in self.bigrams and unigram in self.bigrams[condition]:
                self.A[condition][unigram] = self.bigrams[condition][unigram]
        for unigram, count in self.unigrams.items():
            if unigram in self.A[condition]:
                continue
            self.B[condition][unigram] = count
        self.P_B[condition] = sum(self.P_ML[word] for word in self.B[condition])

    def trainBs(self, bigrams):
        for condition, unigrams in bigrams.items():
            for word in unigrams:
                if condition in self.bigrams and word in self.bigrams[condition]:
                    continue
                self.trainB(condition)

    def bigramMLE(self, word, condition, log=False, verbose=False):
        alpha_steps = None
        steps = ['P(\\texttt{' + condition + '} \mid \\texttt{' + word + '})']
        if condition in self.bigrams_star and word in self.bigrams_star[condition]:
            bigramCount = self.bigrams_star[condition][word]
            unigramCount = self.unigrams[condition]
            conditionalProbability = bigramCount / unigramCount
            steps.append('\\frac{count^{*}(\\texttt{' + condition + '} , \\texttt{' + word + '})}{count(\\texttt{' + condition + '})}')
            steps.append('\\frac{' + str(bigramCount) +  '}{' + str(unigramCount) + '}')
        else:
            if condition not in self.P_B:
                self.trainB(condition)
            
            alpha_numerator = sum(self.bigrams_star[condition].values())
            alpha_denominator = self.unigrams[condition]
            alpha_condition = 1 - alpha_numerator / alpha_denominator
            alpha_steps = [
                '\\alpha_{\\texttt{' + condition + '}}',
                '1 - \\frac{\\Sigma_{w} count^{*}(\\texttt{' + condition + '} , \\texttt{' + word + '})}{count(\\texttt{' + condition + '})}',
                '1 - \\frac{' + str(alpha_numerator) +  '}{' + str(alpha_denominator) + '}',
                alpha_condition
            ]
            *_, wordProbability = self.unigramMLE(word, log=False)
            wordProbabilitiesSum = self.P_B[condition]
            conditionalProbability = alpha_condition * wordProbability / wordProbabilitiesSum
            steps.append('\\alpha_{\\texttt{' + condition + '}} \\times \\frac{P_{ML}(\\texttt{' + word + '})}{\\Sigma_{w \\in B_{\\texttt{' + condition + '}}} P(\\texttt{w})}')
            steps.append(str(alpha_condition) + '\\times \\frac{' + str(wordProbability) +  '}{' + str(wordProbabilitiesSum) + '}')
        
        if log:
            conditionalProbability = math.log(conditionalProbability, 2) if conditionalProbability > 0 else -math.inf
        if log:
            steps = [ '\log_{2} (' + step + ')' for step in steps ]
        steps.append(conditionalProbability)
        if verbose:
            if alpha_steps:
                print(alpha_steps[0] + ' =&\\ ', ' = '.join([ str(step) for step in alpha_steps[1:] ]), ' \\\\')
            print(steps[0] + ' =&\\ ', ' = '.join([ str(step) for step in steps[1:] ]), ' \\\\')
        return steps

## 1.3 QUESTIONS

Please answer the questions below:

### 1. (5 points) How many word types (unique words) are there in the training corpus? Please include the end-of-sentence padding symbol \</s> and the unknown token \<unk>. Do not include the start of sentence padding symbol \<s>.

In [38]:
excluded_unigrams =  {'<s>'}
train_unigrams_with_replacing_without_start = set(train_unigrams_with_replacing.keys()) - excluded_unigrams
print(f'There are {len(train_unigrams_with_replacing_without_start)} word types (unique words) in the training corpus.')

There are 41738 word types (unique words) in the training corpus.


### 2. (5 points) How many word tokens are there in the training corpus? Do not include the start of sentence padding symbol \<s>.

In [39]:
excluded_unigrams =  {'<s>'}

train_unigrams_with_replacing_without_start = sum([ value for word, value in train_unigrams_with_replacing.items() if word not in excluded_unigrams ])
print(f'There are {train_unigrams_with_replacing_without_start} word tokens in the training corpus.')

# test_unigrams_with_replacing_without_start = sum([ value for word, value in test_unigrams_with_replacing.items() if word not in {'<s>'} ])
# print('test unigrams words count', test_unigrams_with_replacing_without_start)

There are 2468210 word tokens in the training corpus.


### 3. (10 points) What percentage of word tokens and word types in the test corpus did not occur in training (before you mapped the unknown words to \<unk> in training and test data)? Please include the padding symbol \</s> in your calculations. Do not include the start of sentence padding symbol \<s>.

In [40]:
excluded_unigrams =  {'<s>'}
test_only_unigrams_without_start = { word: count for word, count in test_only_unigrams.items() if word not in excluded_unigrams }
test_unigrams_without_start = { word: count for word, count in test_unigrams.items() if word not in excluded_unigrams }

print('\\begin{itemize}')
print('\\item Word tokens only in test corpus:', sum(test_only_unigrams_without_start.values()))
print('\\item Word tokens in test corpus', sum(test_unigrams_without_start.values()))
print('\\item Percentage of word tokens only in test corpus:', sum(test_only_unigrams_without_start.values()) / sum(test_unigrams_without_start.values()) * 100)

print('\\item Word types only in test corpus:', len(test_only_unigrams_without_start))
print('\\item Word types in test corpus:', len(test_unigrams_without_start))
print('\\item Percentage of word types only in test corpus:', len(test_only_unigrams_without_start) / len(test_unigrams_without_start) * 100)
print('\\end{itemize}')

\begin{itemize}
\item Word tokens only in test corpus: 46
\item Word tokens in test corpus 2769
\item Percentage of word tokens only in test corpus: 1.6612495485734922
\item Word types only in test corpus: 45
\item Word types in test corpus: 1248
\item Percentage of word types only in test corpus: 3.6057692307692304
\end{itemize}


### 4. (15 points) Now replace singletons in the training data with \<unk> symbol and map words (in the test corpus) not observed in training to \<unk>. What percentage of bigrams (bigram types and bigram tokens) in the test corpus did not occur in training (treat \<unk> as a regular token that has been observed). Please include the padding symbol \</s> in your calculations. Do not include the start of sentence padding symbol \<s>.

In [55]:
excluded_unigrams =  {'<s>'}

test_only_bigrams_with_replacing_no_start = {}
for condition, words in test_bigrams_with_replacing.items():
    if condition not in excluded_unigrams:
        if condition not in train_bigrams_with_replacing:
            for word, count in words.items():
                test_only_bigrams_with_replacing_no_start[(condition, word)] = count
        else:
            for word, count in words.items():
                if word not in train_bigrams_with_replacing[condition]:
                    test_only_bigrams_with_replacing_no_start[(condition, word)] = count

test_bigrams_with_replacing_no_start = {}
for condition, words in test_bigrams_with_replacing.items():
    if condition not in excluded_unigrams:
        for word, count in words.items():
            test_bigrams_with_replacing_no_start[(condition, word)] = count

test_only_bigrams_types_with_replacing_no_start_count = len(test_only_bigrams_with_replacing_no_start)
test_only_bigrams_words_with_replacing_no_start_count = sum(test_only_bigrams_with_replacing_no_start.values())
test_bigrams_types_with_replacing_no_start_count = len(test_bigrams_with_replacing_no_start)
test_bigrams_words_with_replacing_no_start_count = sum(test_bigrams_with_replacing_no_start.values())

print('\\begin{itemize}')
print('\\item Bigrams types only in test corpus:', test_only_bigrams_types_with_replacing_no_start_count)
print('\\item Bigrams types in test corpus:', test_bigrams_types_with_replacing_no_start_count)
print('\\item \\textbf{Percentage of bigrams types only in test corpus}:', test_only_bigrams_types_with_replacing_no_start_count / test_bigrams_types_with_replacing_no_start_count * 100)

print('\\item Bigrams tokens only in test corpus:', test_only_bigrams_words_with_replacing_no_start_count)
print('\\item Bigrams tokens in test corpus:', test_bigrams_words_with_replacing_no_start_count)
print('\\item \\textbf{Percentage of bigrams tokens only in test}:', test_only_bigrams_words_with_replacing_no_start_count / test_bigrams_words_with_replacing_no_start_count * 100)
print('\\end{itemize}')


\begin{itemize}
\item Bigrams types only in test corpus: 595
\item Bigrams types in test corpus: 2300
\item \textbf{Percentage of bigrams types only in test corpus}: 25.869565217391305
\item Bigrams tokens only in test corpus: 597
\item Bigrams tokens in test corpus: 2669
\item \textbf{Percentage of bigrams tokens only in test}: 22.367928062944923
\end{itemize}


### 5. (15 points) Compute the log probability of the following sentence under the three models (ignore capitalization and pad each sentence as described above). Please list all of the parameters required to compute the probabilities and show the complete calculation. Which of the parameters have zero values under each model? Use log base 2 in your calculations. Map words not observed in the training corpus to the \<unk> token.

- I look forward to hearing your reply.

In [178]:
unigramModel = UnigramModel(train_unigrams_with_replacing, ignoredWords={'<s>'})
unigramLogOutput = unigramModel.sentenceMLE('I look forward to hearing your reply .', verbose=True, log=True)
unigramOutput = unigramModel.sentenceMLE('I look forward to hearing your reply .', verbose=True)

\begin{equation}\begin{split}
S =&\ \texttt{I look forward to hearing your reply .} \\
\log_{2} (P(\texttt{i})) =&\  \log_{2} (\frac{count(\texttt{i})}{count()}) = \log_{2} (\frac{7339}{2468210}) = -8.39366593438855  \\
\log_{2} (P(\texttt{look})) =&\  \log_{2} (\frac{count(\texttt{look})}{count()}) = \log_{2} (\frac{613}{2468210}) = -11.97529045258011  \\
\log_{2} (P(\texttt{forward})) =&\  \log_{2} (\frac{count(\texttt{forward})}{count()}) = \log_{2} (\frac{474}{2468210}) = -12.346290467372631  \\
\log_{2} (P(\texttt{to})) =&\  \log_{2} (\frac{count(\texttt{to})}{count()}) = \log_{2} (\frac{53048}{2468210}) = -5.540022976617652  \\
\log_{2} (P(\texttt{hearing})) =&\  \log_{2} (\frac{count(\texttt{hearing})}{count()}) = \log_{2} (\frac{209}{2468210}) = -13.527674584190008  \\
\log_{2} (P(\texttt{your})) =&\  \log_{2} (\frac{count(\texttt{your})}{count()}) = \log_{2} (\frac{1217}{2468210}) = -10.985920263557162  \\
\log_{2} (P(\texttt{reply})) =&\  \log_{2} (\frac{count(\texttt{reply})

In [179]:
bigramModel = BigramModel(train_unigrams_with_replacing, train_bigrams_with_replacing, ignoreWords={})
bigramLogOutput = bigramModel.sentenceMLE('I look forward to hearing your reply .', verbose=True, log=True)

\begin{equation}\begin{split}
S =&\ \texttt{I look forward to hearing your reply .} \\
\log_{2} (P(\texttt{<s>} \mid \texttt{i})) =&\  \log_{2} (\frac{count(\texttt{<s>} , \texttt{i})}{count(<s>)}) = \log_{2} (\frac{2006}{100000}) = -5.639534583824631  \\
\log_{2} (P(\texttt{i} \mid \texttt{look})) =&\  \log_{2} (\frac{count(\texttt{i} , \texttt{look})}{count(i)}) = \log_{2} (\frac{15}{7339}) = -8.93447718627382  \\
\log_{2} (P(\texttt{look} \mid \texttt{forward})) =&\  \log_{2} (\frac{count(\texttt{look} , \texttt{forward})}{count(look)}) = \log_{2} (\frac{34}{613}) = -4.172280422440442  \\
\log_{2} (P(\texttt{forward} \mid \texttt{to})) =&\  \log_{2} (\frac{count(\texttt{forward} , \texttt{to})}{count(forward)}) = \log_{2} (\frac{100}{474}) = -2.2448870591235344  \\
\log_{2} (P(\texttt{to} \mid \texttt{hearing})) =&\  \log_{2} (\frac{count(\texttt{to} , \texttt{hearing})}{count(to)}) = \log_{2} (\frac{6}{53048}) = -13.110048238932082  \\
\log_{2} (P(\texttt{hearing} \mid \texttt{your

In [180]:
smoothBigramModel = SmoothBigramModel(train_unigrams_with_replacing, train_bigrams_with_replacing, ignoreWords={})
smoothBigramLogOutput = smoothBigramModel.sentenceMLE('I look forward to hearing your reply .', verbose=True, log=True)
output = smoothBigramModel.sentenceMLE('I look forward to hearing your reply .', verbose=True)

\begin{equation}\begin{split}
S =&\ \texttt{I look forward to hearing your reply .} \\
\log_{2} (P(\texttt{<s>} \mid \texttt{i})) =&\  \log_{2} (\frac{count^{*}(\texttt{<s>} , \texttt{i}) + 1}{count(<s>) + |V|}) = \log_{2} (\frac{2006 + 1}{100000 + 41739}) = -6.142052348726812  \\
\log_{2} (P(\texttt{i} \mid \texttt{look})) =&\  \log_{2} (\frac{count^{*}(\texttt{i} , \texttt{look}) + 1}{count(i) + |V|}) = \log_{2} (\frac{15 + 1}{7339 + 41739}) = -11.582788837823436  \\
\log_{2} (P(\texttt{look} \mid \texttt{forward})) =&\  \log_{2} (\frac{count^{*}(\texttt{look} , \texttt{forward}) + 1}{count(look) + |V|}) = \log_{2} (\frac{34 + 1}{613 + 41739}) = -10.240859462550434  \\
\log_{2} (P(\texttt{forward} \mid \texttt{to})) =&\  \log_{2} (\frac{count^{*}(\texttt{forward} , \texttt{to}) + 1}{count(forward) + |V|}) = \log_{2} (\frac{100 + 1}{474 + 41739}) = -8.707188259410588  \\
\log_{2} (P(\texttt{to} \mid \texttt{hearing})) =&\  \log_{2} (\frac{count^{*}(\texttt{to} , \texttt{hearing}) + 1}

In [317]:
katzBigramModel = KatzBigramModel(train_unigrams_with_replacing, train_bigrams_with_replacing, ignoreWords={})
katzBigramModel.trainBs(test_only_bigrams)
katzBigramLogOutput = katzBigramModel.sentenceMLE('I look forward to hearing your reply .', verbose=True, log=True)
katzBigramOutput = katzBigramModel.sentenceMLE('I look forward to hearing your reply .', verbose=True)

\begin{equation}\begin{split}
S =&\ \texttt{I look forward to hearing your reply .} \\
\log_{2} (P(\texttt{<s>} \mid \texttt{i})) =&\  \log_{2} (\frac{count^{*}(\texttt{<s>} , \texttt{i})}{count(\texttt{<s>})}) = \log_{2} (\frac{2005.5}{100000}) = -5.639894223622303  \\
\log_{2} (P(\texttt{i} \mid \texttt{look})) =&\  \log_{2} (\frac{count^{*}(\texttt{i} , \texttt{look})}{count(\texttt{i})}) = \log_{2} (\frac{14.5}{7339}) = -8.983386786754767  \\
\log_{2} (P(\texttt{look} \mid \texttt{forward})) =&\  \log_{2} (\frac{count^{*}(\texttt{look} , \texttt{forward})}{count(\texttt{look})}) = \log_{2} (\frac{33.5}{613}) = -4.193654073233009  \\
\log_{2} (P(\texttt{forward} \mid \texttt{to})) =&\  \log_{2} (\frac{count^{*}(\texttt{forward} , \texttt{to})}{count(\texttt{forward})}) = \log_{2} (\frac{99.5}{474}) = -2.2521186283546104  \\
\log_{2} (P(\texttt{to} \mid \texttt{hearing})) =&\  \log_{2} (\frac{count^{*}(\texttt{to} , \texttt{hearing})}{count(\texttt{to})}) = \log_{2} (\frac{5.5}{53048

### 6. (20 points) Compute the perplexity of the sentence above under each of the models.

In [318]:
for output, title in [
    (unigramLogOutput, 'Unigram Model'),
    (bigramLogOutput, 'Bigram Model'),
    (smoothBigramLogOutput, 'Smooth Bigram Model'),
    (katzBigramLogOutput, 'Katz Bigram Model')]:

    *_, M, log_p = output
    l = 1/M * log_p
    pp = math.pow(2, -l)
    print('\\paragraph{%s}' % title)
    print('\\begin{equation}')
    print('\\begin{split}')
    print('M_{S} &= ' + str(M) + ' \\\\')
    print('\\log_{2} (P_{ML}(S)) &= ' + str(log_p) + ' \\\\')
    print('l &= \\frac{\\log_{2} (P_{ML}(S))}{M_{S}} \\\\ &=\ \\frac{' + str(log_p) + '}' + str(M) + ' \\\\ &=\ ' + str(l) +' \\\\')
    print('Perplexity(S) &= 2^{-l} = ' + str(pp) + '\\\\')
    print('\\end{split}')
    print('\\end{equation}')

\paragraph{Unigram Model}
\begin{equation}
\begin{split}
M_{S} &= 9 \\
\log_{2} (P_{ML}(S)) &= -89.74040857086109 \\
l &= \frac{\log_{2} (P_{ML}(S))}{M_{S}} \\ &=\ \frac{-89.74040857086109}9 \\ &=\ -9.971156507873454 \\
Perplexity(S) &= 2^{-l} = 1003.7306831109403\\
\end{split}
\end{equation}
\paragraph{Bigram Model}
\begin{equation}
\begin{split}
M_{S} &= 9 \\
\log_{2} (P_{ML}(S)) &= -inf \\
l &= \frac{\log_{2} (P_{ML}(S))}{M_{S}} \\ &=\ \frac{-inf}9 \\ &=\ -inf \\
Perplexity(S) &= 2^{-l} = inf\\
\end{split}
\end{equation}
\paragraph{Smooth Bigram Model}
\begin{equation}
\begin{split}
M_{S} &= 9 \\
\log_{2} (P_{ML}(S)) &= -97.13956016607362 \\
l &= \frac{\log_{2} (P_{ML}(S))}{M_{S}} \\ &=\ \frac{-97.13956016607362}9 \\ &=\ -10.793284462897068 \\
Perplexity(S) &= 2^{-l} = 1774.607755085189\\
\end{split}
\end{equation}
\paragraph{Katz Bigram Model}
\begin{equation}
\begin{split}
M_{S} &= 9 \\
\log_{2} (P_{ML}(S)) &= -73.38725827637026 \\
l &= \frac{\log_{2} (P_{ML}(S))}{M_{S}} \\ &=\ \f

### 7. (20 points) Compute the perplexity of the entire test corpus under each of the models.Discuss the differences in the results you obtained.

In [319]:
results = []
for sentence in open('test.txt').read().strip().split('\n'):
    # sentence = ' '.join(words[1:-1])
    results.append([ model.sentenceMLE(sentence, log=True) for model in [unigramModel, bigramModel, smoothBigramModel, katzBigramModel] ])


In [320]:
for index, title in enumerate(('unigramModel', 'bigramModel', 'smoothBigramModel', 'katzBigramModel')):
    sumLogP = 0
    sumM = 0
    sumL = 0
    for result in results:
        *_, M, logP = result[index]
        sumLogP += logP
        sumM += M
        sumL += logP/M
    corpusL = 1/sumM * sumLogP
    corpusPP = math.pow(2, -corpusL)

    print('\\paragraph{%s}' % title)
    print('\\begin{equation}')
    print('\\begin{split}')
    print('M_{C} &= \Sigma_{S \in C} M_{S} = ' + str(sumM) + ' \\\\')
    print('\\log_{2} (P_{ML}(C)) &= \Sigma_{S \in C} \\log_{2} (P_{ML}(S)) = ' + str(sumLogP) + ' \\\\')
    print('l_{C} &= \\frac{\\log_{2}(P_{ML}(C))}{M_{C}} \\\\ &=\  \\frac{' + str(sumLogP) + '}' + str(sumM) + ' \\\\ &=\   ' + str(corpusL) +' \\\\')
    print('Perplexity(C) &= 2^{-l_{C}} = ' + str(corpusPP) + '\\\\')
    print('\\end{split}')
    print('\\end{equation}')


\paragraph{unigramModel}
\begin{equation}
\begin{split}
M_{C} &= \Sigma_{S \in C} M_{S} = 2769 \\
\log_{2} (P_{ML}(C)) &= \Sigma_{S \in C} \log_{2} (P_{ML}(S)) = -27965.787053030697 \\
l_{C} &= \frac{\log_{2}(P_{ML}(C))}{M_{C}} \\ &=\  \frac{-27965.787053030697}2769 \\ &=\   -10.099598068989057 \\
Perplexity(C) &= 2^{-l_{C}} = 1097.190308743997\\
\end{split}
\end{equation}
\paragraph{bigramModel}
\begin{equation}
\begin{split}
M_{C} &= \Sigma_{S \in C} M_{S} = 2769 \\
\log_{2} (P_{ML}(C)) &= \Sigma_{S \in C} \log_{2} (P_{ML}(S)) = -inf \\
l_{C} &= \frac{\log_{2}(P_{ML}(C))}{M_{C}} \\ &=\  \frac{-inf}2769 \\ &=\   -inf \\
Perplexity(C) &= 2^{-l_{C}} = inf\\
\end{split}
\end{equation}
\paragraph{smoothBigramModel}
\begin{equation}
\begin{split}
M_{C} &= \Sigma_{S \in C} M_{S} = 2769 \\
\log_{2} (P_{ML}(C)) &= \Sigma_{S \in C} \log_{2} (P_{ML}(S)) = -31072.441669718453 \\
l_{C} &= \frac{\log_{2}(P_{ML}(C))}{M_{C}} \\ &=\  \frac{-31072.441669718453}2769 \\ &=\   -11.221539064542599 \\
Perp

In [304]:
# katzBigramModel2 = KatzBigramModel(train_unigrams_with_replacing, train_bigrams_with_replacing, ignoreWords={})
# katzBigramModel2.trainBs(test_only_bigrams)

A_your = {}
for unigram in train_unigrams_with_replacing:
    if unigram in train_bigrams_with_replacing['your']:
        A_your[unigram] = train_bigrams_with_replacing['your'][unigram]
B_your = {}
for unigram, count in train_unigrams_with_replacing.items():
    if unigram in A_your:
        continue
    B_your[unigram] = count
print(len(A_your), len(B_your))
print(len(A_your) + len(B_your), len(train_unigrams_with_replacing))

P_unigram = { word: katzBigramModel2.unigramMLE(word)[-1] for word in train_unigrams_with_replacing }
print('len(P_unigram)', len(P_unigram))


685 41054
41739 41739
len(P_unigram) 41739


In [305]:
P_A_your = sum(P_unigram[word] for word in A_your)
P_B_your = sum(P_unigram[word] for word in B_your)
print(P_A_your, P_B_your, P_A_your + P_B_your)

0.1180900315784147 0.8819099684217718 1.0000000000001865


In [309]:
P_A_your_model = sum(katzBigramModel2.P_ML[word] for word in katzBigramModel2.A['your'])
P_B_your_model = sum(katzBigramModel2.P_ML[word] for word in katzBigramModel2.B['your'])
print(len(katzBigramModel2.A['your']), len(katzBigramModel2.B['your']))
print(P_A_your_model, P_B_your_model, P_A_your_model + P_B_your_model)

685 41054
0.1180900315784147 0.8819099684217718 1.0000000000001865


In [326]:
from functools import reduce

bigrams2 = [ [(condition, word) in words] for condition, words in test_bigrams.items() ]
bigrams3 = reduce(lambda x, y: x + y, bigrams2, [])
len(bigrams3)

1248

In [264]:
P_B_your

0.6282788401261891